In [11]:
#Input Parameters ---------------------------------------------------------------------------------------------------
s = 4;
a = 8;
demand = 300;
shortcost = 200;


#fixed cost of selecting edge i
f = [273, 152, 169, 164, 210, 264, 123, 178];


#Capacity of edge i in scenario j
u = [[271, 279, 202, 279],
[207, 168, 282, 275],
[169, 282, 172, 273],
[186, 191, 200, 123],
[163, 128, 118, 154],
[236, 156, 274, 146],
[190, 264, 298, 257],
[268, 139, 217, 158]];


#Shipping Cost for edge i
shipcost = [12, 21, 20, 26, 22, 15, 16, 24];


# Probability of scenario s
prob = [0.3, 0.2, 0.4, 0.1];


##Setting the ranges--------------------------------------------------------------------------------------------------
#Setting the number of scenarios
scenarios = range(s)

#Setting the number of arcs
edges = range(a)


#Importing required packages and functions ---------------------------------------------------------------------------
from itertools import product
from mip import Model, xsum, minimize, BINARY, INTEGER
model_2_stage=Model(solver_name='GRB') #Setting the GUROBY solver

# Defining the decision variables -------------------------------------------------------------------------------------
z = [model_2_stage.add_var(var_type=BINARY) for i in edges]
x = [[model_2_stage.add_var(var_type=INTEGER) for j in scenarios] for i in edges]
shortfall = [model_2_stage.add_var(var_type=INTEGER) for i in scenarios]


#Defining the objective function --------------------------------------------------------------------------------------
model_2_stage.objective = minimize(xsum(f[e]*z[e] for e in edges) + xsum(prob[s]*x[e][s]*shipcost[e] for e in edges for s in scenarios) + xsum(prob[s]*shortfall[s]*shortcost for s in scenarios))


#Setting up the constraints --------------------------------------------------------------------------------------------
#Constraint: flow in each arc is nonnegative
for e,s in product(edges, scenarios):
    model_2_stage += x[e][s] >= 0

#Constraint: flow in each arc is less than or equal to the capacity of that arc in the selected scenario
for e,s in product(edges, scenarios):
    model_2_stage += x[e][s] <= u[e][s]*z[e]

#Constraint: the demand node must receive flows equal to the arcs reaching to it 
for s in scenarios:
    model_2_stage += x[6][s] + x[7][s] == demand - shortfall[s]
    
    
#Constraint: flow conservation at node C
for s in scenarios:
    model_2_stage += x[0][s] + x[1][s] == x[3][s]
    
    
#Constraint: flow conservation at node D
for s in scenarios:
    model_2_stage += x[4][s] + x[5][s] == x[2][s]
    
    
#Constraint: flow conservation at node E
for s in scenarios:
    model_2_stage += x[3][s] + x[4][s] == x[6][s]

#Constraint: flow conservation at node F
for s in scenarios:
    model_2_stage += x[5][s] == x[7][s]
    
#Solve the problem ----------------------------------------------------------------------------------------------------    
model_2_stage.optimize()



#Print out solutions and objective function value --------------------------------------------------------------------

#Print objective function value
print(f'Solution found with objective function value: {model_2_stage.objective_value}')

#Print solution
if model_2_stage.num_solutions:
    for e in edges:
        for s in scenarios:
            print(f'arc {e} carries {x[e][s].x} units in scenario: {s}')

Solution found with objective function value: 18081.100000000002
arc 0 carries 186.0 units in scenario: 0
arc 0 carries 191.0 units in scenario: 1
arc 0 carries 200.0 units in scenario: 2
arc 0 carries 123.0 units in scenario: 3
arc 1 carries -0.0 units in scenario: 0
arc 1 carries -0.0 units in scenario: 1
arc 1 carries -0.0 units in scenario: 2
arc 1 carries -0.0 units in scenario: 3
arc 2 carries 114.0 units in scenario: 0
arc 2 carries 109.0 units in scenario: 1
arc 2 carries 100.0 units in scenario: 2
arc 2 carries 177.0 units in scenario: 3
arc 3 carries 186.0 units in scenario: 0
arc 3 carries 191.0 units in scenario: 1
arc 3 carries 200.0 units in scenario: 2
arc 3 carries 123.0 units in scenario: 3
arc 4 carries 4.0 units in scenario: 0
arc 4 carries 73.0 units in scenario: 1
arc 4 carries 98.0 units in scenario: 2
arc 4 carries 134.0 units in scenario: 3
arc 5 carries 110.0 units in scenario: 0
arc 5 carries 36.0 units in scenario: 1
arc 5 carries 2.0 units in scenario: 2
arc